# Quick introduction to AML Data Prep

This notebook quickly walks through the highlights of Azure Machine Learning's Data Prep SDK.

In [1]:
import pandas as pd
from os import path
import azureml.dataprep as dprep
from tempfile import mkdtemp
from azureml.dataprep import f_and, f_not, value

localfilepath = './deepdataapi.txt'

The most common task in data preparation is getting data in. For data science, this is frequently from files. Normally, it requires writing code that looks like this:

In [2]:
dflow = dprep.read_csv(path = localfilepath, separator = '|', skip_rows = 3)
dflow.head(5)

,Name,CompanyName,SalesPerson,EmailAddress,Founded,Last Order,Sales to Date,City,postal_code,latitude,longitude
0,Mr. Seth Juarez,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,21-Feb-73,531,95962473,San Francisco,94122,37.758941,-222.48591
1,Ms Katherine Harding,Vintage Sport Boutique,adventure-works\david8,kendra0@adventure-works.com,5 November 1880,68343,342244200,San Francisco,94122,37.758941,-122.48591
2,Mrs Kami LeMonds,Trendy Department Stores,adventure-works\shu0,donald1@adventure-works.com,8-Oct-43,83287,92839201,San Francisco,94122,37.758941,-122.48591
3,Mr. Andrew Cencini,Sports Merchandise,adventure-works\pamela0,andrew2@adventure-works.com,1915,58533,43569020,SJ,94115,37.782632,-122.432504
4,Mr. Darren Gehring,Journey Sporting Goods,adventure-works\jillian0,darren0@adventure-works.com,28-Jul-32,65744,38783980,San Antonio,94133,47.609722,-122.333056


However, being effective with this approach requires intimate knowledge of the file, the parameters to the read method (whether it be CSV, TXT, Excel, etc), and an iterative approach to trying some params (seeing if they work and trying different ones). Try removing some of the options above to see what happens when params are wrong.

However, the AML Data Prep SDK provides an easier way:

In [3]:
dflow = dprep.auto_read_file(localfilepath)
dflow.head(5)

,Name,CompanyName,SalesPerson,EmailAddress,Founded,Last Order,Sales to Date,City,postal_code,latitude,longitude
0,Mr. Seth Juarez,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,21-Feb-73,531.0,95962473.0,San Francisco,94122.0,37.758941,-222.485910
1,Ms Katherine Harding,Vintage Sport Boutique,adventure-works\david8,kendra0@adventure-works.com,5 November 1880,68343.0,342244200.0,San Francisco,94122.0,37.758941,-122.485910
2,Mrs Kami LeMonds,Trendy Department Stores,adventure-works\shu0,donald1@adventure-works.com,8-Oct-43,83287.0,92839201.0,San Francisco,94122.0,37.758941,-122.485910
3,Mr. Andrew Cencini,Sports Merchandise,adventure-works\pamela0,andrew2@adventure-works.com,1915,58533.0,43569020.0,SJ,94115.0,37.782632,-122.432504
4,Mr. Darren Gehring,Journey Sporting Goods,adventure-works\jillian0,darren0@adventure-works.com,28-Jul-32,65744.0,38783980.0,San Antonio,94133.0,47.609722,-122.333056


Note that the `auto_read_file` does not take any params in this invocation. It also does not specify the type of file. This same call would work for fixed width, text, json, and Excel files. This method uses ML approaches under the covers to develop an understanding of the file to then write a program that will read it. To see what our program understood the file to be, do:

In [4]:
ff = dprep.detect_file_format(localfilepath)
print(ff.file_format)

ParseDelimitedProperties
    separator: '|'
    headers_mode: PromoteHeadersMode.CONSTANTGROUPED
    encoding: FileEncoding.UTF8
    quoting: False
    skip_rows: 3
    skip_mode: SkipMode.GROUPED
    comment: None



`detect_file_format` is being called under the covers by the `auto_read_file`. Here, you can see the parameters that will be used on the read file operation, adding transparency to the call. These params can also be overriden, thus addressing the 2 major complaints about "magical methods":
1. needing insight into what's happening, and
2. needing the ability to override the "magic."

Now that the file is loaded, it's possible to start performing some of the smart operations on it:

In [5]:
profile = dflow.get_profile()
profile

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
Name,FieldType.STRING,Mr. Alexander J. Deborde,Ms. Yuhong Li,66.0,0.0,66.0,0.0,0.0,0.0,,,,,,,,,,,,,,
CompanyName,FieldType.STRING,A Bike Store,Wholesale Parts,66.0,0.0,66.0,0.0,0.0,0.0,,,,,,,,,,,,,,
SalesPerson,FieldType.STRING,adventure-works\david8,adventure-works\shu0,66.0,0.0,66.0,0.0,0.0,0.0,,,,,,,,,,,,,,
EmailAddress,FieldType.STRING,aidan0@adventure-works.com,yuhong1@adventure-works.com,66.0,0.0,66.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Founded,FieldType.STRING,10-May-35,9-Jul-01,66.0,0.0,66.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Last Order,FieldType.DECIMAL,138,99489,66.0,0.0,66.0,0.0,0.0,0.0,138,17432.1,17362,36954,58385.5,73814,95511,99373.2,99489,55340.5,26393.2,6.96603e+08,-0.273553,-0.899927
Sales to Date,FieldType.DECIMAL,2.31478e+06,3.42244e+08,66.0,0.0,66.0,0.0,0.0,0.0,2.31478e+06,9.27195e+06,9.26363e+06,3.00203e+07,5.60074e+07,8.52014e+07,1.09087e+08,3.32982e+08,3.42244e+08,6.32235e+07,5.54891e+07,3.07904e+15,2.76122,10.8135
City,FieldType.STRING,S.A.,San Jose,66.0,0.0,66.0,0.0,0.0,0.0,,,,,,,,,,,,,,
postal_code,FieldType.DECIMAL,94103,94133,66.0,0.0,66.0,0.0,0.0,0.0,94103,94107.1,94107,94115,94122,94133,94133,94133,94133,94122.6,10.3493,107.107,-0.395359,-1.29098
latitude,FieldType.DECIMAL,37.7129,47.6097,66.0,0.0,66.0,0.0,0.0,0.0,37.7129,37.7438,37.7427,37.7589,37.7845,37.7984,47.6097,47.6097,47.6097,38.3725,2.36435,5.59014,3.59906,11.124


Profiling the Dataflow (`dflow`) provides information and statistics about the data that help you make decisions about what operations can/should be performed on the data to get it ready to be consumed for advanced analytics.

It is also possible to look at individual columns and to ask for distribution information about the columns. In the case of the 'City' column, let's look at the frequency distributions:

In [6]:
profile.columns['City'].value_counts

[ValueCountEntry(value='San Francisco', count=15.0),
 ValueCountEntry(value='San Antonio', count=14.0),
 ValueCountEntry(value='SA', count=6.0),
 ValueCountEntry(value='San Jose', count=6.0),
 ValueCountEntry(value='SAN FRANCISCO', count=5.0),
 ValueCountEntry(value='S.A.', count=4.0),
 ValueCountEntry(value='S.J.', count=4.0),
 ValueCountEntry(value='SJ', count=3.0),
 ValueCountEntry(value='SAN JOSE', count=3.0),
 ValueCountEntry(value='S.D.', count=3.0),
 ValueCountEntry(value='San Diego', count=3.0)]

From this information, we see that there are lots of duplicate 'City' names with different spelling, capitalization, and abbreviation. To address this, we use Fuzzy Grouping to create a new column that groups mispelled and inconsistently entered city names:

In [7]:
dflow = dflow.fuzzy_group_column(source_column='City',
                                 new_column_name='Clean City',
                                 similarity_score_column_name= 'SimScore',
                                 similarity_threshold=0.8)

profile = dflow.get_profile()
profile.columns['Clean City'].value_counts

[ValueCountEntry(value='San Antonio', count=24.0),
 ValueCountEntry(value='San Francisco', count=20.0),
 ValueCountEntry(value='San Jose', count=16.0),
 ValueCountEntry(value='San Diego', count=6.0)]

It is also possible to perform more straightforward operations as well like filtering. The next step filters out any rows that are from the city of San Antonio:

In [8]:
dflow = dflow.filter(dprep.col('Clean City') != 'San Antonio')
profile = dflow.get_profile()
profile.columns['Clean City'].value_counts

[ValueCountEntry(value='San Francisco', count=20.0),
 ValueCountEntry(value='San Jose', count=16.0),
 ValueCountEntry(value='San Diego', count=6.0)]

Now that the data set is in a good, clean state, we can start applying transformations that are specific to consumption by ML models like encoders:

In [9]:
dflow = dflow.label_encode(source_column='Clean City', new_column_name='City Label')
dflow.head(5)

,Name,CompanyName,SalesPerson,EmailAddress,Founded,Last Order,Sales to Date,City,Clean City,City Label,SimScore,postal_code,latitude,longitude
0,Mr. Seth Juarez,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,21-Feb-73,531.0,95962473.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-222.485910
1,Ms Katherine Harding,Vintage Sport Boutique,adventure-works\david8,kendra0@adventure-works.com,5 November 1880,68343.0,342244200.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
2,Mrs Kami LeMonds,Trendy Department Stores,adventure-works\shu0,donald1@adventure-works.com,8-Oct-43,83287.0,92839201.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
3,Mr. Andrew Cencini,Sports Merchandise,adventure-works\pamela0,andrew2@adventure-works.com,1915,58533.0,43569020.0,SJ,San Jose,1,0.825086,94115.0,37.782632,-122.432504
4,Mr. Daniel P. Thompson,Travel Sports,adventure-works\pamela0,daniel2@adventure-works.com,10-Oct-50,49186.0,40685181.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910


Once the data is in a good format for modelling, it's important to enforce a contract between the data preparation phase and the modelling phase. What does a contract mean? This might refer to assumptions you have about your dataset, like a range of valid entries. If the contract is broken, it can have consequences on the model that may not be intended. The easiest way to do this is to assert the contract. First, let's profile our data again to determine what our contract might be:

In [10]:
profile = dflow.get_profile()
profile

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
Name,FieldType.STRING,Mr. Andrew Cencini,Ms. Yuhong Li,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
CompanyName,FieldType.STRING,A Bike Store,Wholesale Parts,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
SalesPerson,FieldType.STRING,adventure-works\david8,adventure-works\shu0,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
EmailAddress,FieldType.STRING,andrea1@adventure-works.com,yuhong1@adventure-works.com,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Founded,FieldType.STRING,10-May-35,9-Jul-01,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Last Order,FieldType.DECIMAL,531,98765,42.0,0.0,42.0,0.0,0.0,0.0,531,17267.2,17046,36954,59288,74699,96094,98765,98765,56157.5,26224.1,6.87701e+08,-0.27509,-0.917205
Sales to Date,FieldType.DECIMAL,2.31478e+06,3.42244e+08,42.0,0.0,42.0,0.0,0.0,0.0,2.31478e+06,7.89405e+06,6.66899e+06,2.50516e+07,4.79873e+07,8.43306e+07,1.72392e+08,3.42244e+08,3.42244e+08,6.3477e+07,6.51092e+07,4.2392e+15,2.6567,8.29658
City,FieldType.STRING,S.D.,San Jose,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Clean City,FieldType.STRING,San Diego,San Jose,42.0,0.0,42.0,0.0,0.0,0.0,,,,,,,,,,,,,,
City Label,FieldType.INTEGER,0,2,42.0,0.0,42.0,0.0,0.0,0.0,0,0,0,0,1,1,2,2,2,0.666667,0.721336,0.520325,0.563879,-0.967541


Now that we can see the range of the 'Last Order' column, we can assert what we think should be valid:

In [11]:
dflow = dflow.assert_value(
        columns='Last Order', 
        expression=dprep.f_and(value > 1000, value < 100000),
        error_code='InvalidRange'
    )
dflow.head(5)

,Name,CompanyName,SalesPerson,EmailAddress,Founded,Last Order,Sales to Date,City,Clean City,City Label,SimScore,postal_code,latitude,longitude
0,Mr. Seth Juarez,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,21-Feb-73,"azureml.dataprep.native.DataPrepError(""'Invali...",95962473.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-222.485910
1,Ms Katherine Harding,Vintage Sport Boutique,adventure-works\david8,kendra0@adventure-works.com,5 November 1880,68343,342244200.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
2,Mrs Kami LeMonds,Trendy Department Stores,adventure-works\shu0,donald1@adventure-works.com,8-Oct-43,83287,92839201.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
3,Mr. Andrew Cencini,Sports Merchandise,adventure-works\pamela0,andrew2@adventure-works.com,1915,58533,43569020.0,SJ,San Jose,1,0.825086,94115.0,37.782632,-122.432504
4,Mr. Daniel P. Thompson,Travel Sports,adventure-works\pamela0,daniel2@adventure-works.com,10-Oct-50,49186,40685181.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910


As can be seen from the output, after the assertion our first row fails the data range contract in this Dataflow. Now, it can be filtered out and excluded, or it can be filtered into an error flow and processed there. In this example, we simply filter it out:

In [12]:
dflow = dflow.filter(f_not(dprep.col('Last Order').is_error()))
dflow.head(5)

,Name,CompanyName,SalesPerson,EmailAddress,Founded,Last Order,Sales to Date,City,Clean City,City Label,SimScore,postal_code,latitude,longitude
0,Ms Katherine Harding,Vintage Sport Boutique,adventure-works\david8,kendra0@adventure-works.com,5 November 1880,68343.0,342244200.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
1,Mrs Kami LeMonds,Trendy Department Stores,adventure-works\shu0,donald1@adventure-works.com,8-Oct-43,83287.0,92839201.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
2,Mr. Andrew Cencini,Sports Merchandise,adventure-works\pamela0,andrew2@adventure-works.com,1915,58533.0,43569020.0,SJ,San Jose,1,0.825086,94115.0,37.782632,-122.432504
3,Mr. Daniel P. Thompson,Travel Sports,adventure-works\pamela0,daniel2@adventure-works.com,10-Oct-50,49186.0,40685181.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
4,Mr. Paulo H. Lisboa,Elite Bikes,adventure-works\jillian0,paulo0@adventure-works.com,11 August 1897,87506.0,284357790.0,SAN JOSE,San Jose,1,1.000000,94103.0,37.771437,-122.423892


Once your data is cleaned, prepared, and ready for ML consumption, you have a series of options; the first is simply convert to a data frame (Pandas or Spark) and then consume in an API that can use DataFrames as an input:

In [13]:
df_pandas = dflow.to_pandas_dataframe()
df_pandas

,Name,CompanyName,SalesPerson,EmailAddress,Founded,Last Order,Sales to Date,City,Clean City,City Label,SimScore,postal_code,latitude,longitude
0,Ms Katherine Harding,Vintage Sport Boutique,adventure-works\david8,kendra0@adventure-works.com,5 November 1880,68343.0,342244200.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
1,Mrs Kami LeMonds,Trendy Department Stores,adventure-works\shu0,donald1@adventure-works.com,8-Oct-43,83287.0,92839201.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
2,Mr. Andrew Cencini,Sports Merchandise,adventure-works\pamela0,andrew2@adventure-works.com,1915,58533.0,43569020.0,SJ,San Jose,1,0.825086,94115.0,37.782632,-122.432504
3,Mr. Daniel P. Thompson,Travel Sports,adventure-works\pamela0,daniel2@adventure-works.com,10-Oct-50,49186.0,40685181.0,San Francisco,San Francisco,0,1.000000,94122.0,37.758941,-122.485910
4,Mr. Paulo H. Lisboa,Elite Bikes,adventure-works\jillian0,paulo0@adventure-works.com,11 August 1897,87506.0,284357790.0,SAN JOSE,San Jose,1,1.000000,94103.0,37.771437,-122.423892
5,Mr. Peter Kurniawan,Largest Bike Store,adventure-works\jillian0,peter4@adventure-works.com,25 November 1562,65515.0,97747689.0,S.D.,San Diego,2,0.818806,94133.0,37.798065,-122.406855
6,Ms. Teanna M. Cobb,Another Sporting Goods Company,adventure-works\linda3,teanna0@adventure-works.com,9-Jul-01,37725.0,95659583.0,San Jose,San Jose,1,1.000000,94103.0,37.776775,-122.403305
7,Ms. Jane Clayton,South Bike Company,adventure-works\garrett1,jane1@adventure-works.com,7-Jun-52,66651.0,94831111.0,San Diego,San Diego,2,1.000000,94127.0,37.740654,-122.465389
8,Mr. William J. Conner,Urban Sports Emporium,adventure-works\jose1,william1@adventure-works.com,28-Jul-32,54767.0,93099102.0,San Francisco,San Francisco,0,1.000000,94122.0,37.754114,-122.478330
9,Mr. John L. Colon,Two Wheels Cycle Store,adventure-works\linda3,john14@adventure-works.com,1445,17362.0,87367939.0,SAN FRANCISCO,San Francisco,0,1.000000,94122.0,37.763343,-122.481809


Secondly, it's also possible to save the Dataflow to a `dprep` file to be consumed by other tech like Automated ML:

In [14]:
dflow = dflow.set_name(name="DeepDataAPI")
package_path = path.join(mkdtemp(), "DeepDataAPI.dprep")

print("Saving package to: {}".format(package_path))
package = dprep.Package(arg=dflow)
package.save(file_path=package_path)

Saving package to: /tmp/tmpu6qftv94/DeepDataAPI.dprep


Package
  name: None
  path: /tmp/tmpu6qftv94/DeepDataAPI.dprep
  dataflows: [
    Dataflow {
      name: DeepDataAPI
      steps: 9
    },
  ]